In [ ]:
import panel as pn
pn.extension('plotly')

<div style="display: block;"><center>
<img src="panel_logo_stacked.png" width=250><br>
<h1>Turn any notebook into a deployable dashboard
</h1>
<h2>PyConDE & PyData Berlin 2019</h2>
<br>
<h3>Philipp Rudiger (@philippjfr)
<img src="anaconda-logo.png" width=350>
</center></div>

Do you have a Jupyter notebook with a plot in it?

Or a table, or an image, or an equation?

And maybe you want to explore these things, or share them?

Ok, Panel is for you.

<img src="panel_logo_stacked.png" width=170>

- Panel is relatively new library, but built on Bokeh's solid 1.x release.
- Panel lets all your notebooks double as apps or dashboards.
- Use just about any plotting library, image type, or other objects.
- Develop dashboards in a notebook, deploy, revise, repeat.
- Fully usable with Jupyter *and* without Jupyter installed.

## Demos

- https://panel.pyviz.org/gallery
- https://examples.pyviz.org
- http://localhost:5006
- https://webcam-classifier.pyviz.demo.anaconda.com/

To dive in, let's say we have a dataset to explore, such as this<br>
[UCI ML dataset measuring the environment in a meeting room](http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+).

In [ ]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt

data = pd.read_csv('./occupancy.csv', index_col='date', parse_dates=True)
data.tail()

And we've written some code that smooths a time series and plots it using Matplotlib with outliers highlighted:

In [ ]:
import hvplot.pandas

In [ ]:
def mpl_plot(avg, highlight):
    ax = avg.plot()
    if len(highlight): highlight.plot(style='o', ax=ax)
    fig = ax.get_figure()
    plt.close(fig)
    return fig

def find_outliers(variable='Temperature', window=30, sigma=10, view_fn=mpl_plot):
    avg = data[variable].rolling(window=window).mean()
    residual = data[variable] - avg
    std = residual.rolling(window=window).std()
    outliers = (np.abs(residual) > std * sigma)
    return view_fn(avg, avg[outliers])

We can call the function with parameters and get a plot:

In [ ]:
find_outliers(variable='Temperature', window=20, sigma=10)

It works! But exploring all these parameters by typing Python is slow and tedious. Plus we want our boss, or the boss's boss, to be able to try it out.

Let's make a panel instead:

In [ ]:
pn.interact(find_outliers)

Let's do a bit more work and capture the full range of parameters that can be varied:

In [ ]:
kw = dict(window=(1, 60), variable=sorted(list(data.columns)), sigma=(1, 20))
pn.interact(find_outliers, **kw)

Now that we've explored it, let's share it with the boss:

In [ ]:
pn.interact(find_outliers, **kw).show()

Boss says he doesn't know how to use the dashboard.  

Let's look at it and see how it's made, so we can add some explanatory text:

In [ ]:
i = pn.interact(find_outliers, **kw)
print(i)

Ah, it's just a column of widgets and a plot.

Let's unpack that, rearrange it, add some instructions, and hide one of the widgets that will just confuse the boss:

In [ ]:
text = "<br>\n# Room Occupancy\nSelect the variable, and the time window for smoothing"

p = pn.Row(i[1][0], pn.Column(text, i[0][0], i[0][1], width=400))
p

Once we're happy with that in the notebook, we'll share it again:

In [ ]:
p.show()

In [ ]:
p

Note that even widgets in another notebook cell stay linked:

In [ ]:
i[0][2]

Also note that Panel widgets are reactive, so they will update even if you set the values by hand:

In [ ]:
p

In [ ]:
i[0][0].value = 'Light'

Of course, you don't need to use the magic of `interact`; it's also easy to make widgets and link them up by hand:

In [ ]:
import panel.widgets as pnw

variable  = pnw.RadioButtonGroup(name='variable', value='Temperature', 
                                 options=list(data.columns))
window    = pnw.IntSlider(name='window', value=10, start=1, end=60)

@pn.depends(variable, window)
def reactive_outliers(variable, window):
    return find_outliers(variable, window, 10)

widgets   = pn.Column("<br>\n# Room occupancy", variable, window)
occupancy = pn.Row(reactive_outliers, widgets)

In [ ]:
occupancy

So far we've only used Matplotlib.  

What about other libraries? You can use almost anything!

E.g. [hvPlot](http://hvplot.pyviz.org), a drop-in replacement for Pandas .plot() (and xarray, dask, intake, ...) that gives fully interactive [Bokeh](http://bokeh.pydata.org) plots in panels:

In [ ]:
import hvplot.pandas

def hvplot(avg, highlight):
    return avg.hvplot(height=300, legend=False) * highlight.hvplot.scatter(color='orange', padding=0.1, legend=False)

text2 = "## Room Occupancy\nSelect the variable and the smoothing values"
hvp   = pn.interact(find_outliers, view_fn=hvplot, **kw)
occupancy_app = pn.Column(pn.Row(pn.panel(text2, width=400), hvp[0]), hvp[1])
occupancy_app.servable()

We can bring in other datasets, dozens of other plotting libraries, linked plots, drilling down, big data, etc., but let's call that done for now. 

Can we keep it running as a server indefinitely now?  

Sure, just mark the item to serve with `.servable()` in the notebook, then run `panel serve --show PyDataBerlin2019.ipynb`. 

As you can see, Panel is designed to support your _entire_ data-analysis workflow -- work in a notebook, explore freely, deploy freely, do batch or cron jobs, and never have to rewrite to use your code in a new context.

# Other Features

## Embedding

In [ ]:
occupancy_app.save('test.html', embed=True)

## Templating

```
{% extends base %}

<!-- goes in body -->
{% block postamble %}
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css">
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">

<script src="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/js/materialize.min.js"></script>

<style type="text/css">
  body {
    background-color: var(--jp-layout-color0);
    overflow-y: scroll;
  }
  .nav-wrapper {
    background-color: #2f2f2f;
  }
  .brand-logo {
    height: 75%;
    margin: 8px;
  }
</style>
{% endblock %}

<!-- goes in body -->
{% block contents %}
<header>
  <div class="navbar-fixed">
    <nav class="top-nav">
      <div class="nav-wrapper">
        <a href="#!" class="brand-logo-container">
          <object class="brand-logo" type="image/png" data="http://panel.pyviz.org/_static/logo_horizontal.png"></object>
        </a>
      </div>
    </nav>
  </div>
</header>

<div class="row">

  <div class="col s2">
    <!-- Grey navigation panel -->
  </div>

  <div class="col s8">
    <h1>Custom Material UI Template</h1>
    <p>This is a Panel app with a custom template allowing us to compose multiple Panel objects into a single HTML document.</p>
    <br>
    
    <ul class="collapsible">
      <li>
        <div class="collapsible-header"><i class="material-icons">filter_drama</i>Vega</div>
        <div class="collapsible-body">{{ embed(roots.vega) }}</div>
      </li>
      <li>
        <div class="collapsible-header"><i class="material-icons">place</i>Plotly</div>
        <div class="collapsible-body">{{ embed(roots.plotly) }}</div>
      </li>
      <li>
        <div class="collapsible-header"><i class="material-icons">place</i>HoloViews</div>
        <div class="collapsible-body">{{ embed(roots.holoviews) }}</div>
      </li>
    </ul>
  
  <div class="col s2">
    <!-- Grey navigation panel -->
  </div>
</div>

<script>
  document.addEventListener('DOMContentLoaded', function() {
    var elems = document.querySelectorAll('.collapsible');
    var instances = M.Collapsible.init(elems, {});
  });
</script>

{% endblock %}
```

In [ ]:
template = """
{% extends base %}

<!-- goes in body -->
{% block postamble %}
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css">
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">

<script src="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/js/materialize.min.js"></script>

<style type="text/css">
  body {
    background-color: var(--jp-layout-color0);
    overflow-y: scroll;
  }
  .nav-wrapper {
    background-color: #2f2f2f;
  }
  .brand-logo {
    height: 75%;
    margin: 8px;
  }
</style>
{% endblock %}

<!-- goes in body -->
{% block contents %}
<header>
  <div class="navbar-fixed">
    <nav class="top-nav">
      <div class="nav-wrapper">
        <a href="#!" class="brand-logo-container">
          <object class="brand-logo" type="image/png" data="http://panel.pyviz.org/_static/logo_horizontal.png"></object>
        </a>
      </div>
    </nav>
  </div>
</header>

<div class="row">

  <div class="col s2">
    <!-- Grey navigation panel -->
  </div>

  <div class="col s8">
    <h1>Custom Material UI Template</h1>
    <p>This is a Panel app with a custom template allowing us to compose multiple Panel objects into a single HTML document.</p>
    <br>
    
    <ul class="collapsible">
      <li>
        <div class="collapsible-header"><i class="material-icons">filter_drama</i>Vega</div>
        <div class="collapsible-body">{{ embed(roots.vega) }}</div>
      </li>
      <li>
        <div class="collapsible-header"><i class="material-icons">place</i>Plotly</div>
        <div class="collapsible-body">{{ embed(roots.plotly) }}</div>
      </li>
      <li>
        <div class="collapsible-header"><i class="material-icons">place</i>HoloViews</div>
        <div class="collapsible-body">{{ embed(roots.holoviews) }}</div>
      </li>
    </ul>
  
  <div class="col s2">
    <!-- Grey navigation panel -->
  </div>
</div>

<script>
  document.addEventListener('DOMContentLoaded', function() {
    var elems = document.querySelectorAll('.collapsible');
    var instances = M.Collapsible.init(elems, {});
  });
</script>

{% endblock %}
"""

In [ ]:
imdb = {
  "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/movies.json"},
  "transform": [{
    "filter": {"and": [
      {"field": "IMDB_Rating", "valid": True},
      {"field": "Rotten_Tomatoes_Rating", "valid": True}
    ]}
  }],
  "mark": "rect",
  "width": 600,
  "height": 400,
  "encoding": {
    "x": {
      "bin": {"maxbins":60},
      "field": "IMDB_Rating",
      "type": "quantitative"
    },
    "y": {
      "bin": {"maxbins": 40},
      "field": "Rotten_Tomatoes_Rating",
      "type": "quantitative"
    },
    "color": {
      "aggregate": "count",
      "type": "quantitative"
    }
  },
  "config": {
    "view": {
      "stroke": "transparent"
    }
  }
}

vega = pn.pane.Vega(imdb, width=750, height=425)

# Declare range slider to adjust the color limits
color_lims = pn.widgets.RangeSlider(name='Color limits', start=0, end=125,
                                    value=(0, 40), step=1, width=200)
color_lims.jslink(vega, code={'value': """
target.data.encoding.color.scale = {domain: source.value};
target.properties.data.change.emit()
"""})

# Declare slider to control the number of bins along the x-axis
imdb_bins = pn.widgets.IntSlider(name='IMDB Ratings Bins', start=0, end=125, 
                                 value=60, step=25, width=200)
imdb_bins.jslink(vega, code={'value': """
target.data.encoding.x.bin.maxbins = source.value;
target.properties.data.change.emit()
"""})

# Declare slider to control the number of bins along the y-axis
tomato_bins = pn.widgets.IntSlider(name='Rotten Tomato Ratings Bins', start=0, end=125,
                                   value=40, step=25, width=200)
tomato_bins.jslink(vega, code={'value': """
target.data.encoding.y.bin.maxbins = source.value;
target.properties.data.change.emit()
"""})

vega = pn.Row(vega, pn.Column(color_lims, imdb_bins, tomato_bins))

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

plotly = go.Figure(data=[go.Surface(z=z_data.values)])

plotly.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90));

In [ ]:
import holoviews as hv, param, dask.dataframe as dd
import panel as pn


from colorcet import cm
from holoviews.operation.datashader import rasterize, shade
from holoviews.element.tiles import StamenTerrain
hv.extension('bokeh')

usecols = ['dropoff_x','dropoff_y','pickup_x','pickup_y','dropoff_hour','pickup_hour','passenger_count']
df = dd.read_parquet('/Users/philippjfr/development/datashader/examples/data/nyc_taxi_wide.parq')[usecols].persist()
opts = dict(responsive=True, min_height=500,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
cmaps = ['fire','bgy','bgyw','bmy','gray','kbc']

alpha = pn.widgets.FloatSlider(name='Alpha', start=0, end=1, value=1)
cmap = pn.widgets.Select(name='Colormap', options={c:cm[c] for c in cmaps}, value=cm['fire'])
location = pn.widgets.Select(name='Location', options=['dropoff', 'pickup'], value='dropoff')
hour = pn.widgets.IntRangeSlider(name='Hour', start=0, end=24, value=(0, 24))

def points(ds, loc, h):
    points = hv.Points(ds, [loc+'_x', loc+'_y'], 'dropoff_hour')
    if hour != (0, 24): points = points.select(dropoff_hour=h)
    return points


link = hv.selection.link_selections.instance()

ds = hv.Dataset(df)
tiles = StamenTerrain().apply.opts(alpha=alpha, **opts)
points = ds.apply(points, loc=location, h=hour)
agg = rasterize(points, x_sampling=1, y_sampling=1, width=600, height=400)
shaded = shade(agg, cmap=cmap)
hist = agg.hist('dropoff_hour', adjoin=False)

holoviews = pn.Row(
    pn.Column(pn.WidgetBox(alpha, cmap, location, hour), link(hist)),
    tiles * shaded
)

In [ ]:
tmpl = pn.Template(template)

tmpl.add_panel('vega', vega)
tmpl.add_panel('plotly', plotly)
tmpl.add_panel('holoviews', holoviews)

tmpl.servable()

# Roadmap

* Detailed deployment guides for AWS, GCS, Azure, Heroku etc.
* More examples and expand gallery
* More components (i.e. widgets, panes etc.)
* Add bi-directional support for Jupyter widgets, i.e. include Panel app in Jupyter widget or Jupyter widget in Panel

# Comparison



## Compared to Dash, Panel is:

- Typically much more concise
- More Pythonic (less reliance on HTML/CSS)
- Usable with most plotting libraries, out of the box
- Fully usable from Jupyter notebooks
- More focused on the full life cycle of exploration to deployment

## Compared to Voila+ipywidgets, Panel:

- Is not tied to Jupyter:
   * fully usable from .py, for bigger dashboards
   * deployable without Jupyter installed, for production
- Is more scalable<br>(new connections don't need overhead of a new kernel and can use shared data/computation)
- Makes it simple to designate which output in a notebook is for the dashboard(s)

<img src="http://holoviz.org/assets/holoviz-logo-unstacked.svg" width=350>

Panel and hvPlot are HoloViz projects, like Datashader, HoloViews, GeoViews, Param, Colorcet, ...

See [holoviz.org](https://holoviz.org) to see how everything fits together to solve all your viz needs!

@philippjfr

prudiger@anaconda.com